In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
result_url = "https://www.eoni.org.uk/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards/Elections-2022/NI-Assembly-Election-2022-Result-Sheets"

In [3]:
# EONI is trying to block people from scraping and will return a 403 error if you don't pass a 'conventional' user agent
headers = {
    'user-agent':"User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
}
page = BeautifulSoup(requests.get(result_url, headers=headers).content)

In [4]:
for _p in page.select('.right-column a'):
    if not _p.contents[0].endswith('(XLS)'):
        continue
    else:
        break
_p

<a href="/getmedia/c537e56f-c319-47d1-a2b0-44c90f9aa170/NI-Assembly-Election-2022-Result-Sheet-Belfast-East-XLS">NI Assembly Election 2022 Result Sheet - Belfast East (XLS)</a>

In [5]:
_p.attrs['href']

'/getmedia/c537e56f-c319-47d1-a2b0-44c90f9aa170/NI-Assembly-Election-2022-Result-Sheet-Belfast-East-XLS'

In [6]:
file_url = '/'.join(result_url.split('/')[:3]) + _p.attrs['href']

In [7]:
file_url

'https://www.eoni.org.uk/getmedia/c537e56f-c319-47d1-a2b0-44c90f9aa170/NI-Assembly-Election-2022-Result-Sheet-Belfast-East-XLS'

In [8]:
r = requests.get(file_url, headers=headers)

In [9]:
from io import BytesIO
with requests.get(file_url, headers=headers) as response:
    response.raise_for_status()
    data = BytesIO(response.content)
    df = pd.read_excel(data)


In [10]:
df

,NI Assembly Election,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Stage 12,Unnamed: 6,Unnamed: 7,Unnamed: 8,Date of Poll,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,1,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,Constituency of,NaN,NaN,Belfast East,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Eligible Electorate,70123,Number to be Elected,NaN,5,Invalid Votes,NaN,592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Votes Polled,43840,Total Valid Votes,NaN,43248,% Poll,NaN,62.518717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Stage 1,Stage 2,NaN,Stage 3,NaN,Stage 4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Non Transferable,NaN,0,0


In [11]:
df.iloc[2,9]

62.51871711136147

In [12]:
import re

In [13]:
def get_metadata_from_df(df):
    stage_n_catcher = re.compile(r"^Stage (\d+)")

    metadata = {
        'stage':  int(re.match(stage_n_catcher, df.columns[5]).group(1)),
        # should have been just int(df.columns[5].split()[-1])., but someone insisted on messing up 2017
        'date': df.columns[10],
        'constituency': df.iloc[0, 3],
        'eligible_electorate': df.iloc[1, 3],
        'votes_polled': df.iloc[2, 3],
        'number_to_be_elected': df.iloc[1, 6],
        'total_valid_votes': df.iloc[2, 6],
        'invalid_votes': df.iloc[1, 9],
        'electoral_quota': df.iloc[1, 12]
    }
    return metadata
metadata = get_metadata_from_df(df)
metadata

{'stage': 12,
 'date': datetime.datetime(2022, 5, 5, 0, 0),
 'constituency': 'Belfast East',
 'eligible_electorate': 70123,
 'votes_polled': 43840,
 'number_to_be_elected': 5,
 'total_valid_votes': 43248,
 'invalid_votes': 592,
 'electoral_quota': 7209}

In [14]:
def get_candidates_from_df(df):
    candidates_df = df.iloc[9:29,2:4]
    candidates_df.columns=['candidate_name','candidate_party']
    return candidates_df.replace(0,None).dropna().reset_index(drop=True)
candidates_df = get_candidates_from_df(df)
candidates_df

,candidate_name,candidate_party
0,"ALLEN, Andy",Ulster Unionist Party
1,"BENNETT, Karl George",Progressive Unionist Party of Northern Ireland
2,"BROOKS, David",Democratic Unionist Party - D.U.P.
3,"BUNTING, Joanne",Democratic Unionist Party - D.U.P.
4,"CARSON, Charlotte",SDLP (Social Democratic & Labour Party)
5,"KENNY, Hannah",People Before Profit Alliance
6,"KERR, Lauren Alana",Ulster Unionist Party
7,"LONG, Naomi",Alliance Party
8,"MACNEILL, Eoin",The Workers Party
9,"MCREYNOLDS, Peter",Alliance Party


In [15]:
total = 0

def extract_stage_n_votes(df,n):
    if n == 0:
        return None
    if n < 10:
        row_offset = 9
        col_offset = 4+(2*(n-1))
    else:
        row_offset = 55
        col_offset = 6+(2*(n-10))
        
    return df.iloc[row_offset:row_offset+20,col_offset].reset_index(drop=True)

def extract_stage_n_transfers(df,n):
    """Stage Transfers are associated with the 'next' stage, i.e. stage 1 has no transfers"""
    if n <=1:
        return None
    if n < 10:
        row_offset = 9
        col_offset = 5+(2*(n-2))
    else:
        row_offset = 55
        col_offset = 5+(2*(n-10))
        
    return df.iloc[row_offset:row_offset+20,col_offset].reset_index(drop=True)


In [46]:
extract_stage_n_votes(df,9)

0     6706.96
1           0
2     7091.24
3        7209
4           0
5           0
6           0
7        7209
8           0
9     7413.92
10          0
11    3425.32
12       3393
13          0
14          0
15          0
16          0
17          0
18          0
19          0
Name: Unnamed: 20, dtype: object

In [17]:
stage_df = pd.concat({n:extract_stage_n_transfers(df,n) for n in range(12)}).unstack().T.replace(0,None).dropna(how='all')

In [18]:
stage_df

,2,3,4,5,6,7,8,9,10,11
0,34.2,2.72,None,28.2,27.72,262.84,1054.56,15.72,11,911
1,3.36,1,None,2.12,15.24,-991.72,None,None,None,None
2,12.72,0.12,None,10.12,10.36,329.48,82.2,13.24,1,1869
3,None,None,-44,None,None,None,None,None,None,None
4,23.4,9.24,None,-516.64,None,None,None,None,None,None
5,9.96,24.24,None,42.56,-576.76,None,None,None,None,None
6,15.48,1.36,None,25.72,15.36,66,-1405.92,None,None,None
7,-986,None,None,None,None,None,None,None,None,None
8,3.6,-75.6,None,None,None,None,None,None,None,None
9,791.04,12.84,None,187.92,144.16,18.36,97.6,342,-204.92,None


In [19]:
votes_df = pd.concat({n:extract_stage_n_votes(df,n) for n in range(12)}).unstack().T.replace(0,None).dropna(how='all')

In [20]:
votes_df

,1,2,3,4,5,6,7,8,9,10,11
0,5281,5315.2,5317.92,5317.92,5346.12,5373.84,5636.68,6691.24,6706.96,6717.96,7628.96
1,970,973.36,974.36,974.36,976.48,991.72,None,None,None,None,None
2,6633,6645.72,6645.84,6645.84,6655.96,6666.32,6995.8,7078.0,7091.24,7092.24,8961.24
3,7253,7253,7253,7209,7209,7209,7209,7209,7209,7209,7209
4,484,507.4,516.64,516.64,None,None,None,None,None,None,None
5,500,509.96,534.2,534.2,576.76,None,None,None,None,None,None
6,1282,1297.48,1298.84,1298.84,1324.56,1339.92,1405.92,None,None,None,None
7,8195,7209,7209,7209,7209,7209,7209,7209,7209,7209,7209
8,72,75.6,None,None,None,None,None,None,None,None,None
9,5820,6611.04,6623.88,6623.88,6811.8,6955.96,6974.32,7071.92,7413.92,7209,7209


In [21]:
candidates_df

,candidate_name,candidate_party
0,"ALLEN, Andy",Ulster Unionist Party
1,"BENNETT, Karl George",Progressive Unionist Party of Northern Ireland
2,"BROOKS, David",Democratic Unionist Party - D.U.P.
3,"BUNTING, Joanne",Democratic Unionist Party - D.U.P.
4,"CARSON, Charlotte",SDLP (Social Democratic & Labour Party)
5,"KENNY, Hannah",People Before Profit Alliance
6,"KERR, Lauren Alana",Ulster Unionist Party
7,"LONG, Naomi",Alliance Party
8,"MACNEILL, Eoin",The Workers Party
9,"MCREYNOLDS, Peter",Alliance Party


In [22]:
votes_df

,1,2,3,4,5,6,7,8,9,10,11
0,5281,5315.2,5317.92,5317.92,5346.12,5373.84,5636.68,6691.24,6706.96,6717.96,7628.96
1,970,973.36,974.36,974.36,976.48,991.72,None,None,None,None,None
2,6633,6645.72,6645.84,6645.84,6655.96,6666.32,6995.8,7078.0,7091.24,7092.24,8961.24
3,7253,7253,7253,7209,7209,7209,7209,7209,7209,7209,7209
4,484,507.4,516.64,516.64,None,None,None,None,None,None,None
5,500,509.96,534.2,534.2,576.76,None,None,None,None,None,None
6,1282,1297.48,1298.84,1298.84,1324.56,1339.92,1405.92,None,None,None,None
7,8195,7209,7209,7209,7209,7209,7209,7209,7209,7209,7209
8,72,75.6,None,None,None,None,None,None,None,None,None
9,5820,6611.04,6623.88,6623.88,6811.8,6955.96,6974.32,7071.92,7413.92,7209,7209


In [23]:
results_df = pd.DataFrame(votes_df.values, columns=votes_df.columns, index=candidates_df.candidate_name).dropna(how='all')

In [24]:
results_df[results_df.iloc[:,-1]>=metadata['electoral_quota']].index.values

array(['ALLEN, Andy', 'BROOKS, David', 'BUNTING, Joanne', 'LONG, Naomi',
       'MCREYNOLDS, Peter'], dtype=object)

In [25]:
votes_df

,1,2,3,4,5,6,7,8,9,10,11
0,5281,5315.2,5317.92,5317.92,5346.12,5373.84,5636.68,6691.24,6706.96,6717.96,7628.96
1,970,973.36,974.36,974.36,976.48,991.72,None,None,None,None,None
2,6633,6645.72,6645.84,6645.84,6655.96,6666.32,6995.8,7078.0,7091.24,7092.24,8961.24
3,7253,7253,7253,7209,7209,7209,7209,7209,7209,7209,7209
4,484,507.4,516.64,516.64,None,None,None,None,None,None,None
5,500,509.96,534.2,534.2,576.76,None,None,None,None,None,None
6,1282,1297.48,1298.84,1298.84,1324.56,1339.92,1405.92,None,None,None,None
7,8195,7209,7209,7209,7209,7209,7209,7209,7209,7209,7209
8,72,75.6,None,None,None,None,None,None,None,None,None
9,5820,6611.04,6623.88,6623.88,6811.8,6955.96,6974.32,7071.92,7413.92,7209,7209


In [26]:
cd ~/src/bolster

/Users/bolster/src/bolster


In [27]:
%pip install -e .

Obtaining file:///Users/bolster/src/bolster
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for bolster (pyproject.toml) ... done
  Created wheel for bolster: filename=bolster-0.1.2-py3-none-any.whl size=3319 sha256=8922a01ef4264e761391b8d45b196a5cebce2a9414711ed8700ec496b34b2d19
  Stored in directory: /private/var/folders/9p/y8jwvy1n7v10llkj6r9qtdrc0000gn/T/pip-ephem-wheel-cache-6ib90ht2/wheels/6d/df/1d/5d1fa4d3e4c35449dce1992e04f9dee56e23c4b5348b7107db
Successfully built bolster
  Attempting uninstall: bolster
    Found existing installation: bolster 0.1.2
    Uninstalling bolster-0.1.2:
      Successfully uninstalled bolster-0.1.2
Note: you may need to restart the kernel to use updated packages.


In [28]:
%load_ext autoreload

In [29]:
%autoreload 2

In [47]:
import bolster

In [48]:
from bolster.data_sources.eoni import get_results_from_sheet, get_results_sheets

In [49]:
import logging 
failing_sheets = []
from collections import defaultdict
results = defaultdict(defaultdict)


for sheet_url in get_results_sheets(
    "https://www.eoni.org.uk/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards/Elections-2017/NI-Assembly-Election-2017-Result-Sheets"):
    try:
        data = get_results_from_sheet(sheet_url)
    except ValueError as err:
        failing_sheets.append(sheet_url)
        logging.error(err)

/Users/bolster/src/bolster/src/bolster/data_sources/eoni.py:127: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 127 of the file /Users/bolster/src/bolster/src/bolster/data_sources/eoni.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  page = BeautifulSoup(requests.get(results_sheets_listing_url, headers=_headers).content)


In [50]:
from bolster.data_sources.eoni import get_results

results = get_results(2017)

/Users/bolster/src/bolster/src/bolster/data_sources/eoni.py:127: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 127 of the file /Users/bolster/src/bolster/src/bolster/data_sources/eoni.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  page = BeautifulSoup(requests.get(results_sheets_listing_url, headers=_headers).content)


In [51]:
failing_sheets

[]

In [52]:
results.keys()

dict_keys(['BELFAST EAST', 'BELFAST NORTH', 'BELFAST SOUTH', 'BELFAST WEST', 'EAST ANTRIM', 'EAST LONDONDERRY', 'FERMANAGH AND SOUTH TYRONE', 'FOYLE', 'LAGAN VALLEY', 'MID ULSTER', 'NEWRY AND ARMAGH', 'NORTH ANTRIM', 'NORTH DOWN', 'SOUTH ANTRIM', 'SOUTH DOWN', 'STRANGFORD', 'UPPER BANN', 'WEST TYRONE'])

In [53]:
results['BELFAST EAST']['metadata']

{'stage': 10,
 'date': datetime.datetime(2017, 3, 2, 0, 0),
 'constituency': 'BELFAST EAST',
 'eligible_electorate': 64788,
 'votes_polled': 40828,
 'number_to_be_elected': 5,
 'total_valid_votes': 40357,
 'invalid_votes': 471,
 'electoral_quota': 6727}

In [54]:
results = get_results(2016)

/Users/bolster/src/bolster/src/bolster/data_sources/eoni.py:127: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 127 of the file /Users/bolster/src/bolster/src/bolster/data_sources/eoni.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  page = BeautifulSoup(requests.get(results_sheets_listing_url, headers=_headers).content)


In [55]:
results.keys()

dict_keys(['BELFAST EAST', 'BELFAST NORTH', 'BELFAST SOUTH', 'BELFAST WEST', 'EAST ANTRIM', 'EAST LONDONDERRY', 'FERMANAGH & SOUTH TYRONE', 'FOYLE', 'LAGAN VALLEY', 'MID ULSTER', 'NEWRY & ARMAGH', 'NORTH ANTRIM', 'NORTH DOWN', 'SOUTH ANTRIM', 'SOUTH DOWN', 'STRANGFORD', 'UPPER BANN', 'WEST TYRONE'])

In [56]:
results['BELFAST EAST']['stage_votes']

,1,2,3,4,5,6,7,8,9,10,11,12
0,3047,3057.28,3065.28,3069.57,3176.72,3201.81,3283.15,3286.15,3520.59,3777.94,4756.53,5332.53
1,2183,2184.4,2222.4,2233.29,2283.49,2497.23,2559.32,2893.95,2935.18,3018.46,3050.84,3293.25
2,5538,5311,5311,5311,5311,5311,5311,5311,5311,5311,5311,5311
3,141,141.76,None,None,None,None,None,None,None,None,None,None
4,4230,4401.52,4409,4411.19,4436.26,4456.29,4572.61,4574.61,4756.52,4978.16,5045.16,5461.16
5,887,889.72,891.72,891.99,907.03,911.03,1037.14,1037.14,None,None,None,None
6,78,78,None,None,None,None,None,None,None,None,None,None
7,1099,1100.48,1107.48,1108.26,1113.26,1144.39,1190.57,1192.6,1277.76,None,None,None
8,1772,1776,1786.04,1788.65,1788.71,1816.77,1928.95,1938.95,2082.3,2485.62,2533.87,None
9,631,632.52,634.52,635.15,694.15,717.19,None,None,None,None,None,None


In [60]:
results['BELFAST EAST']['stage_transfers']


,2,3,4,5,6,7,8,9,10,11,12
0,10.28,8,4.29,107.15,25.09,81.34,3,234.44,257.35,978.59,576
1,1.4,38,10.89,50.2,213.74,62.09,334.63,41.23,83.28,32.38,242.41
2,-227,None,None,None,None,None,None,None,None,None,None
3,0.76,-141.76,None,None,None,None,None,None,None,None,None
4,171.52,7.48,2.19,25.07,20.03,116.32,2,181.91,221.64,67,416
5,2.72,2,0.27,15.04,4,126.11,None,-1037.14,None,None,None
6,None,-78,None,None,None,None,None,None,None,None,None
7,1.48,7,0.78,5,31.13,46.18,2.03,85.16,-1277.76,None,None
8,4,10.04,2.61,0.06,28.06,112.18,10,143.35,403.32,48.25,-2533.87
9,1.52,2,0.63,59,23.04,-717.19,None,None,None,None,None


In [61]:
votes_df = results['BELFAST EAST']['stage_votes']
candidates_df = results['BELFAST EAST']['candidates']
metadata = results['BELFAST EAST']['metadata']

In [62]:
votes_df

,1,2,3,4,5,6,7,8,9,10,11,12
0,3047,3057.28,3065.28,3069.57,3176.72,3201.81,3283.15,3286.15,3520.59,3777.94,4756.53,5332.53
1,2183,2184.4,2222.4,2233.29,2283.49,2497.23,2559.32,2893.95,2935.18,3018.46,3050.84,3293.25
2,5538,5311,5311,5311,5311,5311,5311,5311,5311,5311,5311,5311
3,141,141.76,None,None,None,None,None,None,None,None,None,None
4,4230,4401.52,4409,4411.19,4436.26,4456.29,4572.61,4574.61,4756.52,4978.16,5045.16,5461.16
5,887,889.72,891.72,891.99,907.03,911.03,1037.14,1037.14,None,None,None,None
6,78,78,None,None,None,None,None,None,None,None,None,None
7,1099,1100.48,1107.48,1108.26,1113.26,1144.39,1190.57,1192.6,1277.76,None,None,None
8,1772,1776,1786.04,1788.65,1788.71,1816.77,1928.95,1938.95,2082.3,2485.62,2533.87,None
9,631,632.52,634.52,635.15,694.15,717.19,None,None,None,None,None,None


In [63]:
results_df = pd.DataFrame(votes_df.values, columns=votes_df.columns, index=candidates_df.candidate_name).dropna(how='all')
results_df

,1,2,3,4,5,6,7,8,9,10,11,12
candidate_name,,,,,,,,,,,,
"ALLEN, Andrew Joseph David",3047,3057.28,3065.28,3069.57,3176.72,3201.81,3283.15,3286.15,3520.59,3777.94,4756.53,5332.53
"BROWN, Gareth Ross",2183,2184.4,2222.4,2233.29,2283.49,2497.23,2559.32,2893.95,2935.18,3018.46,3050.84,3293.25
"BUNTING, Joanne Sharon",5538,5311,5311,5311,5311,5311,5311,5311,5311,5311,5311,5311
"DOHERTY, Amy Ronella",141,141.76,None,None,None,None,None,None,None,None,None,None
"DOUGLAS, Samuel",4230,4401.52,4409,4411.19,4436.26,4456.29,4572.61,4574.61,4756.52,4978.16,5045.16,5461.16
"GIRVIN, Andrew Craig",887,889.72,891.72,891.99,907.03,911.03,1037.14,1037.14,None,None,None,None
"HOLMES, Joseph Erskine",78,78,None,None,None,None,None,None,None,None,None,None
"HUTTON, Mary Margaret",1099,1100.48,1107.48,1108.26,1113.26,1144.39,1190.57,1192.6,1277.76,None,None,None
"KYLE, Samuel John",1772,1776,1786.04,1788.65,1788.71,1816.77,1928.95,1938.95,2082.3,2485.62,2533.87,None


In [65]:
metadata

{'stage': 13,
 'date': datetime.datetime(2016, 5, 5, 0, 0),
 'constituency': 'BELFAST EAST',
 'eligible_electorate': 65740,
 'votes_polled': 37623,
 'number_to_be_elected': 6,
 'total_valid_votes': 37175,
 'invalid_votes': 448,
 'electoral_quota': 5311}

In [68]:
results_df.iloc[:,-1].sort_values(ascending=False)

candidate_name
DOUGLAS, Samuel                     5461.16
ALLEN, Andrew Joseph David          5332.53
BUNTING, Joanne Sharon                 5311
LONG, Naomi Rachel                     5311
NEWTON, Robert Gray                  4745.5
LYTTLE, Christopher Alexander        3500.9
BROWN, Gareth Ross                  3293.25
MORROW, Timothy William             2636.62
DOHERTY, Amy Ronella                   None
GIRVIN, Andrew Craig                   None
HOLMES, Joseph Erskine                 None
HUTTON, Mary Margaret                  None
KYLE, Samuel John                      None
LAVERY, Jonathan                       None
MCGIMPSEY, Christopher David           None
Ó DONNGHAILE, Niall                    None
ROBINSON, Courtney Louise Taylor       None
WILSON, Neil Alan                      None
Name: 12, dtype: object

In [69]:
metadata['number_to_be_elected']

6

In [70]:
results_df.iloc[:,-1].sort_values(ascending=False)[:metadata['number_to_be_elected']]

candidate_name
DOUGLAS, Samuel                  5461.16
ALLEN, Andrew Joseph David       5332.53
BUNTING, Joanne Sharon              5311
LONG, Naomi Rachel                  5311
NEWTON, Robert Gray               4745.5
LYTTLE, Christopher Alexander     3500.9
Name: 12, dtype: object

In [64]:
results_df[results_df.iloc[:,-1]>=metadata['electoral_quota']].index.values

array(['ALLEN, Andrew Joseph David', 'BUNTING, Joanne Sharon',
       'DOUGLAS, Samuel', 'LONG, Naomi Rachel'], dtype=object)

In [45]:
# Trying to pull the EU election results because it _looks_ in the browser like an xls but it's actually a pdf :(
get_results_from_sheet(
    "https://www.eoni.org.uk/getmedia/2d393c0e-ba88-48da-af69-fec342d34be0/European-Election-2019-Result-Sheet"
)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
elections_page_url = "https://www.eoni.org.uk/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards"
elections_page = BeautifulSoup(
    requests.get(elections_page_url, 
                 headers=headers).content
)

In [ ]:
def get_page(path):
    res = requests.get('https://www.eoni.org.uk'+path, headers=headers)
    page = BeautifulSoup(res.content)
    return page
results_path = "/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards"


for _p in get_page(results_path).select('.right-column a'):
    if _p.contents[0].startswith('Elections'):
        for __p in get_page(_p['href']).select('.right-column h2,h3'):
            if 'NI Assembly' in __p.contents[0]:
                print(__p)
            break

In [ ]:
__p.findChildren()

In [92]:
res = requests.get("https://www.eoni.org.uk/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards/Elections-2022/NI-Assembly-Election-2022-Result-Sheets", headers=headers)
res.raise_for_status()
page = BeautifulSoup(res.content)

In [93]:
page.find('title').contents[0].strip()

'The Electoral Office of Northern Ireland - EONI'

In [102]:
for _p in page.select('.right-column a'):
    if 'XLS' in _p.contents[0]:
        print(_p.attrs['href'])

/getmedia/c537e56f-c319-47d1-a2b0-44c90f9aa170/NI-Assembly-Election-2022-Result-Sheet-Belfast-East-XLS
/getmedia/5c7b6481-d2df-465c-b4e1-0127bccc68f4/NI-Assembly-Election-2022-Result-Sheet-Belfast-North-XLS
/getmedia/d51ea833-f54d-48da-8b07-f8f09d8bcbce/NI-Assembly-Election-2022-Result-Sheet-Belfast-South-XLS
/getmedia/4d54d1e0-28df-434e-b463-e97907696812/NI-Assembly-Election-Results-Belfast-West
/getmedia/e0dc2d16-0d39-4764-9491-8cb6bab5fee1/NI-Assembly-Election-2022-Result-Sheet-East-Antrim
/getmedia/167a3545-1af1-4940-8ef4-bac93b76e839/NI-Assembly-Election-2022-Result-Sheet-East-Londonderry
/getmedia/3a262add-4e6d-4892-8c82-834ee1474cec/NI-Assembly-Election-2022-Result-Sheet-Fermanagh-South-Tyrone-XLS
/getmedia/41d30317-16e5-4841-9c10-c20f9820c63c/NI-Assembly-Election-2022-Result-Sheet-Foyle-XLS
/getmedia/96df0fbc-e4a9-488a-9eb8-b8484ef1f522/NI-Assembly-Election-2022-Result-Sheet-Lagan-Valley-XLS
/getmedia/87fe5cf5-deff-4a5f-a4dd-5dd762704008/NI-Assembly-Election-Results-Mid-Ulster


In [95]:
from bolster.data_sources.eoni import get_results

In [96]:
results = get_results(2022)

In [97]:
results.keys()

dict_keys(['belfast east', 'belfast north', 'belfast south', 'belfast west', 'east antrim', 'east londonderry', 'fermanagh and south tyrone', 'foyle', 'lagan valley', 'mid ulster', 'newry and armagh', 'north antrim', 'north down', 'south antrim', 'south down', 'upper bann', 'west tyrone'])

In [98]:
get_results_from_sheet('https://www.eoni.org.uk/getmedia/46e5db94-44d3-4f7e-935b-3c7d6a77c593/NI-Assembly-Election-2022-Result-Sheet-Strangford-XLS')

{'candidates':                   candidate_name                          candidate_party
 0              ARMSTRONG, Kellie                           Alliance Party
 1                COOPER, Stephen                      TUV - No Sea Border
 2                  HARVEY, Harry       Democratic Unionist Party - D.U.P.
 3                 HOUSTON, Conor  SDLP (Social Democratic & Labour Party)
 4                      KING, Ben                              Independent
 5             MACARTNEY, Maurice             Green Party Northern Ireland
 6                 MATHISON, Nick                           Alliance Party
 7                MCGIVERN, Róisé                                Sinn Féin
 8   MCILVEEN, Michelle Elizabeth       Democratic Unionist Party - D.U.P.
 9                  NESBITT, Mike                    Ulster Unionist Party
 10                 SMITH, Philip                    Ulster Unionist Party
 11             WEIR, Peter James       Democratic Unionist Party - D.U.P.,
 'stage_vo

In [103]:
from bolster.data_sources.eoni import *

In [104]:
list(find_xls_links_in_page(get_page('/Elections/Election-results-and-statistics/Election-results-and-statistics-2003-onwards/Elections-2022/NI-Assembly-Election-2022-Result-Sheets')))

['https://www.eoni.org.uk/getmedia/c537e56f-c319-47d1-a2b0-44c90f9aa170/NI-Assembly-Election-2022-Result-Sheet-Belfast-East-XLS',
 'https://www.eoni.org.uk/getmedia/5c7b6481-d2df-465c-b4e1-0127bccc68f4/NI-Assembly-Election-2022-Result-Sheet-Belfast-North-XLS',
 'https://www.eoni.org.uk/getmedia/d51ea833-f54d-48da-8b07-f8f09d8bcbce/NI-Assembly-Election-2022-Result-Sheet-Belfast-South-XLS',
 'https://www.eoni.org.uk/getmedia/4d54d1e0-28df-434e-b463-e97907696812/NI-Assembly-Election-Results-Belfast-West',
 'https://www.eoni.org.uk/getmedia/e0dc2d16-0d39-4764-9491-8cb6bab5fee1/NI-Assembly-Election-2022-Result-Sheet-East-Antrim',
 'https://www.eoni.org.uk/getmedia/167a3545-1af1-4940-8ef4-bac93b76e839/NI-Assembly-Election-2022-Result-Sheet-East-Londonderry',
 'https://www.eoni.org.uk/getmedia/3a262add-4e6d-4892-8c82-834ee1474cec/NI-Assembly-Election-2022-Result-Sheet-Fermanagh-South-Tyrone-XLS',
 'https://www.eoni.org.uk/getmedia/41d30317-16e5-4841-9c10-c20f9820c63c/NI-Assembly-Election-2022

In [107]:
set(sorted({'belfast east', 'foyle', 'east antrim', 'upper bann', 'south down', 'fermanagh and south tyrone',
             'west tyrone', 'north antrim', 'belfast west', 'mid ulster', 'belfast south', 'belfast north',
             'east londonderry', 'lagan valley', 'strangford', 'north down', 'south antrim', 'newry and armagh'}))

{'belfast east',
 'belfast north',
 'belfast south',
 'belfast west',
 'east antrim',
 'east londonderry',
 'fermanagh and south tyrone',
 'foyle',
 'lagan valley',
 'mid ulster',
 'newry and armagh',
 'north antrim',
 'north down',
 'south antrim',
 'south down',
 'strangford',
 'upper bann',
 'west tyrone'}

In [108]:
pd.get_pdf("https://www.eoni.org.uk/getmedia/1a44c401-6422-4e18-a0c8-e2aea541078c/ni_assembly_election_2011_-_BE")

AttributeError: module 'pandas' has no attribute 'get_pdf'